In [11]:
import pandas as pd
import glob
from openai import OpenAI
import configparser
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import json
from parallel_dataset import *
import sys
import os
import inspect

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
chat_data_folder_path = 'output_chat_data/'
parallel_folder = 'output_parallel_data/'
df_chat_summary = pd.read_csv(chat_data_folder_path+'chat_preprocess_summary.csv')
df_chat_summary

,username,word_count_median,punctuation_count_avg,vocabulary_diversity,emoji_avg,lexical_density,readability_score
0,U0,8.0,1.06,0.40,0.135802,0.38,81.48
1,U1,7.0,0.95,0.43,0.128713,0.43,97.20
2,U2,3.0,0.40,0.46,0.111111,0.48,110.44
3,U3,6.0,0.80,0.40,0.057377,0.41,93.51
4,U4,7.0,0.93,0.36,0.207407,0.37,98.45
5,U5,6.0,0.53,0.43,0.000000,0.40,96.77
6,U6,6.0,1.29,0.43,0.075758,0.47,101.46
7,U7,4.0,0.62,0.55,0.184211,0.41,109.54
8,U8,10.0,1.50,0.42,0.023810,0.27,79.46
9,U9,9.0,2.00,0.43,0.075000,0.38,84.65


### Create Parallel Datasets (Mistral-Medium and GPT-4 based)

In [29]:
csv_files = glob.glob(chat_data_folder_path + '/*.csv')
usernames= []
median_word_count_list = []
num_sentences = []
original_num_sentences = []
for file in csv_files:
    # Check if the file ends with 'chat_llm.csv'
    if file.endswith('chat_llm.csv'):
        # Read the CSV file into a dataframe
        df = pd.read_csv(file)
        original_num_sentences.append(df.shape[0])
        username = df['username'].iloc[0]
        
        median_word_count = df_chat_summary[df_chat_summary.username == username]['word_count_median']
        
        df = df[df['word_count']>median_word_count.iloc[0]][['messageID','username','message']]
        df = df.reset_index(drop=True)
        input_sentences=list(df['message'])
        
        usernames.append(username)
        median_word_count_list.append(median_word_count.iloc[0])
        num_sentences.append(len(input_sentences))

        
        # get_mistral_medium_output(input_sentences,username)
        # get_gpt_4_output(input_sentences,username)


In [30]:
df_parallel = pd.DataFrame({
    'username': usernames,
    'median_word_threshold': median_word_count_list,
    'num_original_sentences' : original_num_sentences,
    'num_filtered_sentences': num_sentences
})
df_parallel['percentage_filtered'] = round((df_parallel['num_filtered_sentences'] / df_parallel['num_original_sentences']) * 100,2)

df_parallel

,username,median_word_threshold,num_original_sentences,num_filtered_sentences,percentage_filtered
0,U9,9.0,40,17,42.50
1,U8,10.0,42,18,42.86
2,U5,6.0,125,57,45.60
3,U2,3.0,153,70,45.75
4,U3,6.0,122,57,46.72
5,U4,7.0,135,56,41.48
6,U6,6.0,66,29,43.94
7,U1,7.0,101,43,42.57
8,U0,8.0,81,38,46.91
9,U7,4.0,76,34,44.74


In [5]:
df_parallel.to_csv(parallel_folder + 'filtered_sentences_summary.csv',index=None)

### Postprocess files

In [85]:
postprocess_files()

/Users/bojansimoski/dev/eur/llm-evaluate/parallel_dataset.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bojansimoski/dev/eur/llm-evaluate/parallel_dataset.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bojansimoski/dev/eur/llm-evaluate/parallel_dataset.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### Generate Keyword Details

In [9]:
# get_keyword_details()

### Generate Context

In [12]:
# generate_context()